In [ ]:
# importing some libraries
import numpy as np
import os
import helper

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, transforms
import imageio
from PIL import Image
import glob

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import Adam, SGD
import torch.nn.functional as F

##check later for repeated packages
from skimage.transform import resize
from skimage.color import rgb2gray


from google.colab import files

In [ ]:

##import train x and y and test x and y

train_x = torch.load("train_x_new.pt")

train_y = torch.load("train_y_new.pt")

val_x = torch.load("val_x_new.pt")

val_y = torch.load("val_y_new.pt")

In [ ]:


class Dataprep(Dataset):
    def __init__(self, inputarr, outputarr):
        
        self.listinput = inputarr
        self.listoutput = outputarr
        
    def __len__(self):
        return len(self.listinput)
    
    
    def __getitem__(self, index):
        
        if torch.is_tensor(index):
            index = index.tolist()
            
        return self.listinput[index], self.listoutput[index]
    


class NN(nn.Module):   
    def __init__(self):
        super(NN, self).__init__()

        self.conv1 = nn.Conv2d(1, 5, 10)
        self.pool1 = nn.MaxPool2d(3,5)
        self.conv2 = nn.Conv2d(5, 5, 3)
        self.pool2 = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(6440, 500)
        self.fc2 = nn.Linear(500, 2)
        
        
        
    def forward(self, x):
        first = self.conv1(x)
        
        second = F.relu(first)

        third = self.pool1(second)

        fourth = self.conv2(third)

        fifth = F.relu(fourth)

        sixth = self.pool2(fifth)

        sixth = sixth.flatten(1)

        seventh = self.fc1(sixth)
        
        eighth = F.relu(seventh)
        
        nineth = self.fc2(eighth)
        
        
        return eighth

#torch.manual_seed(8) #for reproducibility once I get a good result

net = NN()
loss_function = nn.CrossEntropyLoss()
learning_rate = 0.0001

optimizer = torch.optim.SGD(net.parameters(), learning_rate,momentum=0.5)
batch_sz = 1
            
train_dataset = Dataprep(train_x,train_y)
train_loader = DataLoader(dataset = train_dataset, shuffle = True, batch_size = batch_sz)

test_dataset = Dataprep(val_x, val_y)
test_loader = DataLoader(dataset=test_dataset)

#torch.manual_seed(8) #for reproducibility once I get a good result

total = 0
x = list([])

for epoch in range(3):
    for i, o in train_loader:
        optimizer.zero_grad()
        outputs = net(i)
        loss = loss_function(outputs, o.long())
        loss.backward()
        optimizer.step()
    total +=1 
    print(total)

    #plot the loss func if you want
    #x.append(loss.item())
    #plt.plot(x)


total = 0
correct = 0

with torch.no_grad():
    for data in test_loader:
        i, o =data
        outputs = net(i)
        _, predicted = torch.max(outputs.data, 1)
        total += o.size(0)
        correct += (predicted == o).sum().item()
        
print(100*correct / total)    

        

1
2
3
86.04651162790698
